In [1]:
import os

In [2]:
%pwd

'd:\\AI ML Project\\oral-lesion-detection-ai\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\AI ML Project\\oral-lesion-detection-ai'

In [28]:
import tensorflow as tf

In [29]:
model = tf.keras.models.load_model("artifacts/training/model.keras")

In [30]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model : Path
    training_data : Path
    all_params : dict
    mlflow_tracking_uri : str
    params_image_size : list
    params_batch_size : int


In [31]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [32]:
class ConfigurationManager:
    def __init__(self, config_file_path = CONFIG_FILE_PATH, params_file_path = PARAMS_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])


    def get_evaluation_config(self) -> EvaluationConfig:
        training = self.config.training
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "Oral Cancer Image Dataset")
        params = self.params

        evaluation_config = EvaluationConfig(
            path_of_model=Path(training.trained_model_path),
            training_data=training_data,
            all_params=params,
            mlflow_tracking_uri="https://dagshub.com/shivaganesh-koppishetty/oral-lesion-detection-ai.mlflow",
            params_image_size=params.IMAGE_SIZE,
            params_batch_size=params.BATCH_SIZE
        )

        return evaluation_config


In [33]:
import tensorflow as tf
import mlflow
import mlflow.keras
from pathlib import Path
from urllib.parse import urlparse

In [ ]:
class Evaluation:
    def __init__(self, config : EvaluationConfig):
        self.config = config

    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./225,
            validation_split = 0.30
        )

        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            interpolation = "bilinear"
        )


        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory = self.config.training_data,
            subset = "validation",
            shuffle = False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path:Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = self.model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0] , "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)


    def log_into_mlflow(self):
        mlflow.set_tracking_uri("https://dagshub.com/shivaganesh-koppishetty/oral-lesion-detection-ai.mlflow")
        mlflow.set_experiment("oral-lesion-experiment")

        if mlflow.active_run():
            mlflow.end_run()

        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run(run_name="Oral Lesion Eval Run"):
            mlflow.log_params(self.config.all_params)    
            mlflow.log_metrics({"loss": self.score[0], "accuracy": self.score[1]})

            if tracking_url_type_store != "file":
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")


       
        

In [37]:
try:
    config = ConfigurationManager()
    evaluation_config = config.get_evaluation_config()
    evaluation = Evaluation(evaluation_config)
    evaluation._valid_generator()
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
    raise e   

[2025-08-01 16:03:27,433: INFO: common: yaml file : config\config.yaml loaded succesfully]
[2025-08-01 16:03:27,435: INFO: common: yaml file : params.yaml loaded succesfully]
[2025-08-01 16:03:27,436: INFO: common: created directory at: artifacts]
Found 282 images belonging to 2 classes.
Found 282 images belonging to 2 classes.
18/18 ━━━━━━━━━━━━━━━━━━━━ 29s 2s/step - Accuracy: 0.8238 - loss: 4.6137
[2025-08-01 16:03:56,540: INFO: common: json file saved at: scores.json]


2025/08/01 16:03:57 INFO mlflow.tracking.fluent: Experiment with name 'oral-lesion-experiment' does not exist. Creating a new experiment.
2025/08/01 16:03:59 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2025/08/01 16:04:14 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'VGG16Model' already exists. Creating a new version of this model...
2025/08/01 16:04:42 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: VGG16Model, version 2
Created version '2' of model 'VGG16Model'.


🏃 View run Oral Lesion Eval Run at: https://dagshub.com/shivaganesh-koppishetty/oral-lesion-detection-ai.mlflow/#/experiments/1/runs/fadaebb379ae45bd961ee840fbd9ab21
🧪 View experiment at: https://dagshub.com/shivaganesh-koppishetty/oral-lesion-detection-ai.mlflow/#/experiments/1
